In [ ]:
!pip install Cython
!pip install pycocotools
!pip install torchvision

In [1]:
%matplotlib inline
import os
import torch
import numpy as np
from torchvision import datasets, models
from torchvision import transforms as Transforms
import torch.utils.data as data
from PIL import Image
from pycocotools.coco import COCO
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from tqdm.notebook import tqdm, trange

In [27]:
class CocoaDataset(data.Dataset):

    def __init__(self, root, annotation,transforms = None):
        self.root = root
        self.transforms = transforms
        self.coco = COCO(annotation)
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __getitem__(self, index):
        # COCO file
        coco = self.coco
        # Image ID
        img_id = self.ids[index]
        # Get Annotation from COCO
        ann_ids = coco.getAnnIds(imgIds = img_id)
        # target COCO Annotation file for an image
        coco_ann = coco.loadAnns(ann_ids)
        # Path for the input image
        path = coco.loadImgs(img_id)[0]['file_name']
        # Open the input Image
        img = Image.open(os.path.join(self.root, path)) 
        # Number of object in the image
        num_obj = len(coco_ann)

        ###
        # bounding boxes for objects
        # In COCO format bbox = [xmin, ymin, width, height]
        # In Pytorch the input should be [xmin, ymin, xmax, ymax]
        ###
        boxes = []
        labels = []
        areas = []
        iscrowds = []
        for ann in coco_ann:
            x, y, w, h = ann['bbox']
            boxes.append([x, y, x+w, y+h])
            areas.append(ann['area'])
            labels.append(ann['category_id'])
            iscrowds.append(ann['iscrowd'])
        boxes = torch.as_tensor(boxes, dtype = torch.float32)
        areas = torch.as_tensor(areas, dtype = torch.float32)
        iscrowds = torch.as_tensor(iscrowds, dtype = torch.int64)
        #print(labels)
        labels = torch.as_tensor(labels, dtype = torch.int64)
        img_id = torch.tensor([img_id])

        # Annotation in dict format and values in dtype = tensor
        annotation = {}
        annotation['boxes'] = boxes
        annotation['labels'] = labels
        annotation['iscrowd'] = iscrowds
        annotation['area'] = areas
        annotation['image_id'] = img_id
        #print(annotation)
        if self.transforms is not None:
            img = self.transforms(img)
        
        return img, annotation
    
    def __len__(self):
        return len(self.ids)


In [28]:
# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))

In [29]:
train_data_dir = 'cocoa_diseases_dataset/train/images'
train_coco_ann = 'cocoa_diseases_dataset/train/cocoa_diseases_train.json'
transforms = Transforms.Compose([
    Transforms.ToTensor()
])
dataset = CocoaDataset(train_data_dir, train_coco_ann, transforms)
dataLoarder = data.DataLoader(dataset, batch_size = 1, shuffle = True, num_workers=2,
                                          collate_fn=collate_fn)


loading annotations into memory...
Done (t=0.24s)
creating index...
index created!


In [30]:
num_classes = 3
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


In [31]:
def train(model,dataloader, lr = 0.1, epoches = 20):
    model.train()
    optimizer = torch.optim.SGD(model.parameters(), lr = lr)
    #loss_func = nn.CrossEntropyLoss()
    for epoch in trange(epoches):
        for images, annotations in tqdm(dataloader):
            images = list(img for img in images)
            #annotations = [{k: v for k, v in t.items()} for t in annotations]
            #print(annotations)
            loss_dict = model(images, annotations)
            losses = sum(loss for loss in loss_dict.values())
            #losses = torch.sum()
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

In [32]:
train(model= model, dataloader= dataLoarder, epoches=2)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/297 [00:00<?, ?it/s]

IndexError: Target 284885 is out of bounds.